In [122]:
import numpy as np
import pandas as pd

%matplotlib inline

#For Preprocessing
import os
import email
from email import policy
from sklearn.model_selection import train_test_split


Preprocessing of Data

In [2]:
#Making a dataframe using the 'labels' file

#initialize all the needed paths
path_string = 'C:\\Users\\ASUS\\Desktop\\PythonJupyter'
labels = r'C:\Users\ASUS\Desktop\PythonJupyter\labels'

df = pd.read_csv(labels, sep='\s+', names=['label','file'])    #read the file
df['file'] = df['file'].map(lambda x: x.lstrip('..'))          #remove the '..' from the email path
df['file'] = df['file'].str.replace('/','\\')                  #replace the forward slashes with backslashes to make it uniformed with the rest of the path
df['filepath'] = path_string + df['file'].astype(str)          #add the root path to the given paths to form the absolute paths
df


,label,file,filepath
0,ham,\data\000\000,C:\Users\ASUS\Desktop\PythonJupyter\data\000\000
1,spam,\data\000\001,C:\Users\ASUS\Desktop\PythonJupyter\data\000\001
2,spam,\data\000\002,C:\Users\ASUS\Desktop\PythonJupyter\data\000\002
3,ham,\data\000\003,C:\Users\ASUS\Desktop\PythonJupyter\data\000\003
4,spam,\data\000\004,C:\Users\ASUS\Desktop\PythonJupyter\data\000\004
...,...,...,...
37817,spam,\data\126\017,C:\Users\ASUS\Desktop\PythonJupyter\data\126\017
37818,spam,\data\126\018,C:\Users\ASUS\Desktop\PythonJupyter\data\126\018
37819,spam,\data\126\019,C:\Users\ASUS\Desktop\PythonJupyter\data\126\019
37820,spam,\data\126\020,C:\Users\ASUS\Desktop\PythonJupyter\data\126\020


In [3]:
#This function is to traverse through the files to get to the email files and then extract the email body and add them to a list

def preprocess():
    
    path = r'C:\Users\ASUS\Desktop\PythonJupyter\data'
    filepaths = []  #contains the filepaths of the extracted email body
    messages = [] #contains the extracted email body
    
    for dirpath, dirnames, filenames in os.walk(path):  #dirnames = folder names from 000-126
            
        for file in filenames:

            try:
                filepath = os.path.join(dirpath, file) #C:\Users\ASUS\Desktop\PythonJupyter\data\000\000

                data = open(filepath, encoding="latin-1", errors="ignore")
                data_read = data.read()

                #extract only the main content of the email not including the email header
                message = email.message_from_string(data_read, policy=email.policy.default)  
                body = message.get_body(('plain',))
                if body:
                    body = body.get_content()
                    filepaths.append(filepath)
                    messages.append(body)

            except LookupError:
                body = "can't decode"
                
            
    return filepaths, messages
    

In [4]:
filepaths, messages = preprocess()

In [5]:
index_list = []

#index list contains all the indeces in the dataframe that have successfully extracted email body
for path in filepaths:
    index = df[df['filepath'] == path].index[0]
    index_list.append(index)
    
print(len(index_list)) #- 33,528
print(df.shape) #- 37,822


33528
(37822, 3)


In [6]:
#deleting all the rows in the dataframe whose indeces are not found in the index_list/removing those that wasn't able to extract email body

df.drop(df[~ df.index.isin(index_list)].index, inplace=True)
df


,label,file,filepath
0,ham,\data\000\000,C:\Users\ASUS\Desktop\PythonJupyter\data\000\000
1,spam,\data\000\001,C:\Users\ASUS\Desktop\PythonJupyter\data\000\001
2,spam,\data\000\002,C:\Users\ASUS\Desktop\PythonJupyter\data\000\002
3,ham,\data\000\003,C:\Users\ASUS\Desktop\PythonJupyter\data\000\003
4,spam,\data\000\004,C:\Users\ASUS\Desktop\PythonJupyter\data\000\004
...,...,...,...
37817,spam,\data\126\017,C:\Users\ASUS\Desktop\PythonJupyter\data\126\017
37818,spam,\data\126\018,C:\Users\ASUS\Desktop\PythonJupyter\data\126\018
37819,spam,\data\126\019,C:\Users\ASUS\Desktop\PythonJupyter\data\126\019
37820,spam,\data\126\020,C:\Users\ASUS\Desktop\PythonJupyter\data\126\020


In [7]:
#Add another column in the dataframe for the emails or the messages that were extracted from the emails
#messages list contains all the email bodies that were extracted

df['message'] = messages 
df

,label,file,filepath,message
0,ham,\data\000\000,C:\Users\ASUS\Desktop\PythonJupyter\data\000\000,The mailing list I queried about a few weeks a...
1,spam,\data\000\001,C:\Users\ASUS\Desktop\PythonJupyter\data\000\001,\n ...
2,spam,\data\000\002,C:\Users\ASUS\Desktop\PythonJupyter\data\000\002,Academic Qualifications available from prestig...
3,ham,\data\000\003,C:\Users\ASUS\Desktop\PythonJupyter\data\000\003,Greetings all. This is to verify your subscri...
4,spam,\data\000\004,C:\Users\ASUS\Desktop\PythonJupyter\data\000\004,try chauncey may conferred the luscious not co...
...,...,...,...,...
37817,spam,\data\126\017,C:\Users\ASUS\Desktop\PythonJupyter\data\126\017,Great News Expec ted!\n\nInfinex Ventures Inc....
37818,spam,\data\126\018,C:\Users\ASUS\Desktop\PythonJupyter\data\126\018,The OIL sector is going crazy. This is our wee...
37819,spam,\data\126\019,C:\Users\ASUS\Desktop\PythonJupyter\data\126\019,http://vdtobj.docscan.info/?23759301\n\nSuffer...
37820,spam,\data\126\020,C:\Users\ASUS\Desktop\PythonJupyter\data\126\020,U N I V E R S I T Y D I P L O M A S\n\nDo you...


In [8]:
#removed alphanumeric and special characters from the content
df['content'] = df['message'].str.replace('[^\w\s]','',regex=True)


In [9]:
#Remove stop words from the email messages

text = open(r'C:\Users\ASUS\Desktop\PythonJupyter\stop_words.txt')
text_read = text.read()

df['content'] = df['content'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in text_read))
df.head(20)

,label,file,filepath,message,content
0,ham,\data\000\000,C:\Users\ASUS\Desktop\PythonJupyter\data\000\000,The mailing list I queried about a few weeks a...,the mailing list i queried weeks ago running i...
1,spam,\data\000\001,C:\Users\ASUS\Desktop\PythonJupyter\data\000\001,\n ...,luxury watches buy your own rolex for only 219...
2,spam,\data\000\002,C:\Users\ASUS\Desktop\PythonJupyter\data\000\002,Academic Qualifications available from prestig...,academic qualifications prestigious nonacc red...
3,ham,\data\000\003,C:\Users\ASUS\Desktop\PythonJupyter\data\000\003,Greetings all. This is to verify your subscri...,greetings this verify subscription plan9fans l...
4,spam,\data\000\004,C:\Users\ASUS\Desktop\PythonJupyter\data\000\004,try chauncey may conferred the luscious not co...,chauncey conferred luscious continued tonsillitis
5,ham,\data\000\005,C:\Users\ASUS\Desktop\PythonJupyter\data\000\005,"It's quiet. Too quiet. Well, how about a str...",its quiet too quiet well straw poll how plan9 ...
6,ham,\data\000\006,C:\Users\ASUS\Desktop\PythonJupyter\data\000\006,It's working here. I have departed almost tot...,its working i departed totally bell labs recom...
7,spam,\data\000\007,C:\Users\ASUS\Desktop\PythonJupyter\data\000\007,From NBC Today Show:\n\nIt's the look everyone...,from nbc today show its diet theyre beaches ma...
8,spam,\data\000\008,C:\Users\ASUS\Desktop\PythonJupyter\data\000\008,The OIL sector is going crazy. This is our wee...,the oil sector going crazy this weekly gift ge...
9,spam,\data\000\009,C:\Users\ASUS\Desktop\PythonJupyter\data\000\009,Little magic. Perfect weekends.\nhttp://othxu....,little magic perfect weekends httpothxurzfzwiw...


In [10]:
#Make a copy of df because I want to save all the original information
#The new dataframe will be cleaned and will only contain the label and content
new_df = df.copy()
new_df.drop(['file','filepath','message'], axis = 1, inplace = True)

#Split the dataset into training set and testing set

#Train set is folder 0 - 70 while test set is folders 71-126
#print(df[df['filepath']=='C:\\Users\\ASUS\\Desktop\\PythonJupyter\\data\\070\\299']) #Last file in train set - index 21299
#print(df[df['filepath']=='C:\\Users\\ASUS\\Desktop\\PythonJupyter\\data\\071\\000']) #21300
#print(df[df['filepath']=='C:\\Users\\ASUS\\Desktop\\PythonJupyter\\data\\126\\021']) #37821, the last file in dataset

train_set = new_df.iloc[:21299,:]       #has 21,299 rows
test_set = new_df.iloc[21299:,:]     #has 12,229 rows


In [11]:
new_df

,label,content
0,ham,the mailing list i queried weeks ago running i...
1,spam,luxury watches buy your own rolex for only 219...
2,spam,academic qualifications prestigious nonacc red...
3,ham,greetings this verify subscription plan9fans l...
4,spam,chauncey conferred luscious continued tonsillitis
...,...,...
37817,spam,great news expec infinex ventures inc infx pri...
37818,spam,the oil sector going crazy this weekly gift ge...
37819,spam,httpvdtobjdocscaninfo23759301 suffering pain d...
37820,spam,u n i v e r s i t y d i p l o m a s do prosper...


In [31]:
#Make a dictionary of all unique words in the training set

import heapq

def get_common_words(text):
    
    frequencies = {}

    for content in text:
        #tokenize the content for each row
        tokens = content.split()
        #check each word from the tokenized words and add them to the dictionary if they still aren't there
        for token in tokens:
            if token not in frequencies.keys():   #if the word is not in the dictionary keys, add them with their value as 1
                frequencies[token] = 1
            else:
                frequencies[token] += 1           #if the word is already there, just increase their frequency (value)

    #extract only the 10000 most common words or those words with highest frequencies
    common_words = heapq.nlargest(10000, frequencies, key=frequencies.get)

    return common_words



In [39]:
train_set_words = train_set['content']
common_words = get_common_words(train_set_words)
print(common_words)

['i', 'the', 'a', '1', 'received', 'from', 'to', 'this', 'list', 'you', '2', 'subject', 'board', 'l', '3', 'price', 'email', 'if', 'we', '10', 'dmdxpsy1psycharizonaedu', 'nil', 'date', 'help', 'dont', 'company', '0', '0700', 'message', 'send', 'file', 'adobe', 'in', 'it', 'hi', 'dmdx', 'v', 'wrote', 'program', '2001', 'im', 'work', 'well', '1998', 'crustl', '5', 'problem', 'и', 'c', 'thanks', 're', 'esmtp', 'university', 'contenttype', 'number', 'x', '2000', 'good', 'wed', 'professional', 'gold', 'windows', 'mail', '15', '8', '4', 'our', '6', 'textplain', 'microsoft', 'mimeversion', 'dec', 'code', 'sender', 'news', 'replyto', 'office', 'r', 'thu', 's', '2002', 'm', 'messageid', 'system', 'for', 'dmdxownerpsy1psycharizonaedu', 'ms', 'handyboard', 'find', 'hb', '7', 'inc', 'no', 'в', 'may', 'is', 'campaign', 'set', 'current', '1999', 'data', 'pro', 'mon', 'fax', 'tue', 'g', 'at', 'text', 'studies', 'charsetusascii', 'unsubscribe', 'fri', 'address', 'and', 'save', 'people', 'today', 'file

Creating the feature matrices



In [13]:
#create train set for spam and train set for ham
train_set_spam = train_set[train_set['label'] == 'spam']    #has 13,015 rows
train_set_ham = train_set[train_set['label'] == 'ham']      #has 8,284 rows

In [14]:
#create feature matrices for spam and ham training sets

#First, create a matrix for the spam training set
spam_lists = []  #this is where we continuously add the vectors from each email

for message in train_set_spam['content']:  #tokenize each email content in the content column of the trainset of spam
    msg_tokens = message.split()
    email_vector = []                 #create an empty list dedicated to the results of that email
    for word in common_words:         #traverse through the common_words list
        if word in msg_tokens:        #check if the word from the common_words can be found in the email content
            email_vector.append(1)    #if the word is found in the email content, we put 1 in the vector of the email content
        else:
            email_vector.append(0)    #else we put 0
            
    spam_lists.append(email_vector)    #we add the vector to the spam_lists (which is like a list of lists)
            
#convert the spam_lists into a real matrix
spam_matrix = np.array(spam_lists)


#Second, create a matrix for the ham training set
ham_lists = []  

for msg in train_set_ham['content']:  
    ms_tokens = msg.split()
    msg_vector = []                 
    for wd in common_words:         
        if wd in ms_tokens:        
            msg_vector.append(1)    
        else:
            msg_vector.append(0)    
            
    ham_lists.append(msg_vector)    
            
ham_matrix = np.array(ham_lists)



In [15]:
print(spam_matrix.shape)
print(ham_matrix.shape)
ham_matrix

(13015, 10000)
(8284, 10000)


array([[1, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [16]:
#See the matrix in dataframe form
spam_df = pd.DataFrame(spam_matrix, columns=common_words)
spam_df

,i,the,a,1,received,from,to,this,list,you,...,pike,madrid,effect,interpretations,discusses,raymond,pray,nslt,miniaturization,supperlattice
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13010,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
13011,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13013,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
ham_df = pd.DataFrame(ham_matrix, columns=common_words)
ham_df

,i,the,a,1,received,from,to,this,list,you,...,pike,madrid,effect,interpretations,discusses,raymond,pray,nslt,miniaturization,supperlattice
0,1,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8279,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8280,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8281,1,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8282,1,1,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


Computing the Priors

In [18]:
#computer for the prior probabilities for spam and ham

spam_prior = len(train_set_spam)/len(train_set)   #13015/21299
ham_prior = len(train_set_ham)/len(train_set)   #8284/21299
print(spam_prior)
print(ham_prior)

0.6110615521855486
0.38893844781445136


Computing the Likelihood of each word

In [19]:

spam_dict = {}
ham_dict = {}

# getting the occurences of each word in all spam emails
for line in train_set_spam['content']:
    split_words = line.split()
    for word in split_words:
        if word not in spam_dict.keys():
            spam_dict[word] = 1
        else:
            spam_dict[word] += 1
            
            
#getting the occurences of each word in all ham emails           
for line2 in train_set_ham['content']:
    split_words2 = line2.split()
    for word2 in split_words2:
        if word2 not in ham_dict.keys():
            ham_dict[word2] = 1
        else:
            ham_dict[word2] += 1
            


In [30]:
#computing for the probabilities of each word in the spam and ham emails

spam_probabilities = {}   #742,596
ham_probabilities = {}    #132,0101

total_spam = sum(list(spam_dict.values()))
total_ham = sum(list(ham_dict.values()))
        
for line in train_set_spam['content']:
    words = line.split()
    for w in words:
        spam_probability = (spam_dict[w] + 1) / (total_spam + len(common_words))
        spam_probabilities[w] = spam_probability

for line2 in train_set_ham['content']:
    words2 = line2.split()
    for w2 in words2:
        ham_probability = (ham_dict[w2] + 1) / (total_ham + len(common_words))
        ham_probabilities[w2] = ham_probability
        
print('spam_probabilities: ',list(spam_probabilities.items())[:20])
print('\nham_probabilities: ',list(ham_probabilities.items())[:20])


spam_probabilities:  [('luxury', 0.00016980618162879457), ('watches', 0.0005595626522130077), ('buy', 0.00038063936016119053), ('your', 0.0015248367182505178), ('own', 0.0002643962022676533), ('rolex', 0.0007601390815195032), ('for', 0.0007624183591252589), ('only', 0.00035556730649787856), ('219', 0.0001789232920518171), ('cartier', 0.0002586980082532642), ('bvlgari', 0.00024274306501297478), ('frank', 0.00017436473684030582), ('muller', 0.0002359052321957079), ('patek', 0.00023362595458995225), ('philippe', 0.00023362595458995225), ('vacheron', 0.00022678812177268537), ('constantin', 0.00022678812177268537), ('a', 0.010521145428167997), ('lange', 0.000229067399378441), ('sohne', 0.00016980618162879457)]

ham_probabilities:  [('the', 0.007598550593985367), ('mailing', 0.00039691985007844354), ('list', 0.003815489343156019), ('i', 0.024059050259003175), ('queried', 1.9456855396002135e-06), ('weeks', 0.00017640882225708604), ('ago', 0.0002827729650885644), ('running', 0.0008885297297507

Classifying the emails

In [97]:
import math

def classify_emails(email):
    log_p_spam = 0
    log_p_ham = 0
    words = set(email.split())    #to only get unique words
    for w in words:
        if w in spam_probabilities.keys():
            log_p_spam += math.log(spam_probabilities[w])   #find the probability of that word being spam
        if w in ham_probabilities.keys():
            log_p_ham += math.log(ham_probabilities[w])     ##find the probability of that word being ham
            
        
    alpha  = abs(max(log_p_spam, log_p_ham))  
    spam_probability = math.exp(log_p_spam + alpha) * spam_prior
    ham_probability  = math.exp(log_p_ham  + alpha) * ham_prior
    
    spam_class = spam_probability / (spam_probability + ham_probability)
    ham_class = ham_probability / (spam_probability + ham_probability)

    return spam_class, ham_class


Testing the classifier

In [98]:
#testing some email messages from the dataset
#the message is supposed to be ham (ham ../data/001/001)

spam_class, ham_class = classify_emails("if you want a bof, schedule it and see what happens.i'm a little curious myself")
print(spam_class, ham_class)

0.43986827625088565 0.5601317237491144


In [100]:
#Another example, the message is labeled as spam (spam ../data/000/009)
spam_class, ham_class = classify_emails("Little magic. Perfect weekends.http://othxu.rzfzwiwwfoeh9rr2wrr2errr.myrciaid.com/?jef")
print(spam_class, ham_class)

0.6110615521855486 0.38893844781445136


In [104]:
#Another example, the message is labeled as spam (spam ../data/000/001)
spam_class, ham_class = classify_emails("LUXURY  WATCHES - BUY YOUR OWN ROLEX  FOR ONLY  $219! Rolex :: Cartier ::  Bvlgari  ::  Frank Muller  ::  Patek  Philippe :: Vacheron Constantin A.  Lange  & Sohne  ::  Audemars  Piguet  :: Jaeger-Lecoultre  ::  IWC  :: Officine  Panerai Breitling ::  Omega ::  Tag  HeuerExapmle:ROLEX  Full 18K Gold  Daytona  for MEN -  only  $269! - Fast delivery -  The lowest prices  in the world -  Worldwide shipping Visit  our shop at:http://vvhvjk130.sewandeatone.com")
print(spam_class, ham_class)

0.9999996934162327 3.065837673042645e-07


In [103]:
#Own examples
spam_class, ham_class = classify_emails("Hello, it's me. Please respond to my call")
print(spam_class, ham_class)

spam_class, ham_class = classify_emails("Answer this form and get a chance to win 100,000!! Answer now")
print(spam_class, ham_class)

0.03268581836935137 0.9673141816306486
0.9269219725359716 0.07307802746402829


Performance Evaluation

In [95]:
#In here, we evaluate the performance of our function in classifying emails 

def evaluate_performance(spam_set, ham_set):
    TP = 0 #correctly classified spam emails (true positive)
    FN = 0 #number of misclassified spam emails (false negative)
    
    for email in spam_set:
        spam_prob, ham_prob = classify_emails(email)
        if spam_prob > 0.5:    #if the spam probability is greater than 50%, it is classified as spam
            TP += 1            #count it as true negative
        else:
            FN += 1          #else, count it as false negative
            
    FP = 0 #number of misclassified ham emails (false positive)
    TN = 0 #correctly classified ham email (true negative)
    
    for email in ham_set:
        spam_prob, ham_prob = classify_emails(email)
        if ham_prob > 0.5:    #if the ham probability is greater than 50%, it is classified as ham
            TN += 1           #count it as true negative
        else:
            FP += 1           #else, count it as false positive
        
        
    precision = TP / (TP + FP)
    recall    = TP / (TP + FN)
    accuracy  = (TN + TP) / (TN + TP + FP + FN)
    
    return precision, recall, accuracy
    
    

In [120]:
#evaluate the model using train set
evaluate_performance(train_set_spam['content'], train_set_ham['content'])

(0.4304913678618858, 0.9782713664896185, 0.48819193389361004)

In [121]:
#divide test set into spam and ham
test_set_spam = test_set[test_set['label'] == 'spam']    #7856 rows
test_set_ham = test_set[test_set['label'] == 'ham']   #4373

#evaluate the model using test set
evaluate_performance(test_set_spam['content'], test_set_ham['content'])

(0.433130081300813, 0.9746169677566888, 0.5347943413198135)